In [2]:
import json
import pandas as pd 

f = open('train/day-0.json')
data = json.load(f)

In [3]:
df = pd.json_normalize(data)

In [10]:
# данные о печах
ovens = pd.json_normalize(data['ovens'])
ovens

,start_temp,working_temps,operations
0,1220,"[960, 1030, 1190, 1100, 1230, 1070, 1110, 1080...","[prokat, kovka]"
1,990,"[980, 1190, 1030, 1100, 990]","[otzhig, kovka]"
2,1220,"[970, 1060, 1220]",[otzhig]
3,1020,"[1060, 1100, 1140, 1240, 1150, 1020]",[kovka]
4,1020,"[970, 1100, 1070, 980, 1180, 990, 1020]",[otzhig]
...,...,...,...
417,1240,"[960, 1040, 1180, 1070, 1240]",[kovka]
418,1050,"[1030, 1160, 1040, 980, 1110, 1020, 990, 1050]",[prokat]
419,1140,"[1060, 1230, 1040, 1170, 990, 1140]",[kovka]
420,950,"[1030, 1190, 1160, 1070, 980, 950]",[prokat]


In [4]:
# данные о сериях
series = pd.json_normalize(data['series'])
series

,temperature,operations
0,1080,"[{'name': 'nagrev', 'timing': 245}, {'name': '..."
1,970,"[{'name': 'nagrev', 'timing': 62}, {'name': 'o..."
2,1080,"[{'name': 'nagrev', 'timing': 76}, {'name': 'k..."
3,980,"[{'name': 'nagrev', 'timing': 96}, {'name': 'o..."
4,980,"[{'name': 'nagrev', 'timing': 82}, {'name': 'p..."
...,...,...
2589,960,"[{'name': 'nagrev', 'timing': 281}, {'name': '..."
2590,1060,"[{'name': 'nagrev', 'timing': 248}, {'name': '..."
2591,1000,"[{'name': 'nagrev', 'timing': 182}, {'name': '..."
2592,1090,"[{'name': 'nagrev', 'timing': 204}, {'name': '..."


In [22]:
# excel 
import numpy as np

series = pd.read_excel('ПланОМД.xlsm', sheet_name='Серии')
series
series = series.drop(columns=['статус', 'Сплав', 'Гр. спл.', 'Бойки'])
series

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев,подогрев,подогрев.1,подогрев.2,подогрев.3
0,"34-8993-18№1,2",2.0,950.0,115-105,105-90,90-90,NaN,NaN
1,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
2,"34-8993-18№3,4",2.0,950.0,115-105,105-90,90-90,NaN,NaN
3,NaN,NaN,NaN,2ч+15м ковка,1ч+15м ковка,1ч+15м ковка,NaN,NaN
4,"34-8993-18№5,6",2.0,950.0,115-105,105-90,90-90,NaN,NaN
...,...,...,...,...,...,...,...,...
343,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
344,2513-18,2.0,960.0,220-150,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,3ч+15м прокат,NaN,NaN,NaN,NaN
346,2523-18,2.0,960.0,220-150,NaN,NaN,NaN,NaN


In [23]:

for i in range(0, series.shape[0], 2):
    series.iloc[i, 3:8] = series.iloc[i+1, 3:8]
    # series = series.drop(labels = [i+1],axis = 0)
    
series

series_clear = series[series.index % 2 != 1]
series_clear
series_reindex = series_clear.reset_index(drop=True)
series_reindex
series_with_op = series_reindex
series_with_op
for i in [1, 4, 7, 10, 13]:
    if i==1:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op['нагрев'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op['нагрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op['нагрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    if i==4:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op['подогрев'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op['подогрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op['подогрев'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==7:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.1'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.1'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.1'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==10:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.2'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.2'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.2'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
    elif i==13:
        series_with_op[f'нагрев_operation_{i}'] = series_with_op[f'подогрев.3'].apply(lambda x: int(x[0]) if type(x) == str else np.NaN)
        series_with_op[f'ковка_operation_{i+1}'] = series_with_op[f'подогрев.3'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='ковка' else np.NaN)
        series_with_op[f'прокат_operation_{i+2}'] = series_with_op[f'подогрев.3'].apply(lambda x: float(1/4) if type(x) == str and x.split()[-1]=='прокат' else np.NaN)
        
series_with_op
series_with_op_clear = series_with_op.drop(columns=['нагрев', 'подогрев', 'подогрев.1', 'подогрев.2', 'подогрев.3'])
series_with_op_clear

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# --
series_with_op_clear.describe()
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    object 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    int64  
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [25]:
series_with_op_clear['нагрев_operation_1'] = series_with_op_clear['нагрев_operation_1'].astype(float)
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    object 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    float64
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [26]:
series_with_op_clear['№слитка-индекс-часть'] = series_with_op_clear['№слитка-индекс-часть'].astype('string')
series_with_op_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №слитка-индекс-часть  174 non-null    string 
 1   кол. в садке, шт      174 non-null    float64
 2   Температура           174 non-null    float64
 3   нагрев_operation_1    174 non-null    float64
 4   ковка_operation_2     123 non-null    float64
 5   прокат_operation_3    48 non-null     float64
 6   нагрев_operation_4    84 non-null     float64
 7   ковка_operation_5     84 non-null     float64
 8   прокат_operation_6    0 non-null      float64
 9   нагрев_operation_7    66 non-null     float64
 10  ковка_operation_8     66 non-null     float64
 11  прокат_operation_9    0 non-null      float64
 12  нагрев_operation_10   12 non-null     float64
 13  ковка_operation_11    12 non-null     float64
 14  прокат_operation_12   0 non-null      float64
 15  нагрев_operation_13   0

In [27]:
series_done = series_with_op_clear
series_done

,№слитка-индекс-часть,"кол. в садке, шт",Температура,нагрев_operation_1,ковка_operation_2,прокат_operation_3,нагрев_operation_4,ковка_operation_5,прокат_operation_6,нагрев_operation_7,ковка_operation_8,прокат_operation_9,нагрев_operation_10,ковка_operation_11,прокат_operation_12,нагрев_operation_13,ковка_operation_14,прокат_operation_15
0,"34-8993-18№1,2",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"34-8993-18№3,4",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"34-8993-18№5,6",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"34-8993-18№7,8",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"34-8993-18№9,10",2.0,950.0,2.0,0.25,NaN,1.0,0.25,NaN,1.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,34-8973-18,1.0,1100.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2483-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2493-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2513-18,2.0,960.0,3.0,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
series_done.to_csv('series_done.csv')

In [187]:
test_df = pd.read_csv('ovens_done.csv', index_col=0)
test_df

,name,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [76]:
blank = pd.read_excel('ПланОМД.xlsm', sheet_name='Бланк')
blank

,Бланк ОМД,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,Печь,Т-ры,Опер.,от Ø,Нач. т-ра,Посл. т-ра,23-0ч.,NaN,NaN,NaN,...,NaN,21-22ч.,NaN,NaN,NaN,22-23ч.,NaN,NaN,NaN,23-0ч.
1,Печь №2,"960, 950, 830, 860, 1060",ковка,NaN,1060,NaN,"3AL-2,5V, 15143-18",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,[440-385]Б.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1060'C 1шт.,NaN,NaN,NaN,...,NaN,NaN,K,K,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Печь №3,"960, 950, 860, 820, 1000",ковка,NaN,960,NaN,"Э110, 714-18-1слиток",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,[440-390]Б.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,960'C 1шт.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN


In [86]:
type(blank.iloc[2][0])

float

In [90]:
float('nan')

nan

In [232]:
blank_non_NaN = blank.dropna(how='all')[['Бланк ОМД', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4']]
blank_non_NaN

,Бланк ОМД,Unnamed: 1,Unnamed: 2,Unnamed: 4
0,Печь,Т-ры,Опер.,Нач. т-ра
1,Печь №2,"960, 950, 830, 860, 1060",ковка,1060
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
7,Печь №3,"960, 950, 860, 820, 1000",ковка,960
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN
13,Печь №10,"960, 950, 840, 870",ковка,950
14,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN


In [245]:
# in [1, 7, 13, 19, 25, 31, 37]
blank_non_NaN = blank_non_NaN[((blank_non_NaN.index-1)%6==0)&(blank_non_NaN.index<=37)]
blank_non_NaN

,Бланк ОМД,Unnamed: 1,Unnamed: 2,Unnamed: 4
1,Печь №2,"960, 950, 830, 860, 1060",ковка,1060
7,Печь №3,"960, 950, 860, 820, 1000",ковка,960
13,Печь №10,"960, 950, 840, 870",ковка,950
19,Печь №15,"960, 950, 840, 870",ковка,950
25,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг",980
31,Печь №21,"1100, 1060, 960, 950","ковка, прокат",950
37,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат",1060


In [246]:
blank_reset_index = blank_non_NaN.reset_index(drop=True)
blank_reset_index

,Бланк ОМД,Unnamed: 1,Unnamed: 2,Unnamed: 4
0,Печь №2,"960, 950, 830, 860, 1060",ковка,1060
1,Печь №3,"960, 950, 860, 820, 1000",ковка,960
2,Печь №10,"960, 950, 840, 870",ковка,950
3,Печь №15,"960, 950, 840, 870",ковка,950
4,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг",980
5,Печь №21,"1100, 1060, 960, 950","ковка, прокат",950
6,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат",1060


In [247]:
blank_reset_index.columns=['name', 'temp', 'operation', 'start_temp']
blank_reset_index

,name,temp,operation,start_temp
0,Печь №2,"960, 950, 830, 860, 1060",ковка,1060
1,Печь №3,"960, 950, 860, 820, 1000",ковка,960
2,Печь №10,"960, 950, 840, 870",ковка,950
3,Печь №15,"960, 950, 840, 870",ковка,950
4,Печь №20,"1060, 980, 940, 960, 750","прокат, отжиг",980
5,Печь №21,"1100, 1060, 960, 950","ковка, прокат",950
6,Печь №22,"1100, 1060, 980, 960, 950","ковка, прокат",1060


In [248]:
def foo1(x):
    return list(x.replace(',', '').split())
blank_reset_index['temp'] = blank_reset_index['temp'].apply(foo1)
blank_reset_index['operation'] = blank_reset_index['operation'].apply(foo1)
# list('1, 2, 3'.replace(',', '').split())
blank_reset_index

,name,temp,operation,start_temp
0,Печь №2,"[960, 950, 830, 860, 1060]",[ковка],1060
1,Печь №3,"[960, 950, 860, 820, 1000]",[ковка],960
2,Печь №10,"[960, 950, 840, 870]",[ковка],950
3,Печь №15,"[960, 950, 840, 870]",[ковка],950
4,Печь №20,"[1060, 980, 940, 960, 750]","[прокат, отжиг]",980
5,Печь №21,"[1100, 1060, 960, 950]","[ковка, прокат]",950
6,Печь №22,"[1100, 1060, 980, 960, 950]","[ковка, прокат]",1060


In [249]:
# enc.categories_

In [250]:
blank_reset_index

,name,temp,operation,start_temp
0,Печь №2,"[960, 950, 830, 860, 1060]",[ковка],1060
1,Печь №3,"[960, 950, 860, 820, 1000]",[ковка],960
2,Печь №10,"[960, 950, 840, 870]",[ковка],950
3,Печь №15,"[960, 950, 840, 870]",[ковка],950
4,Печь №20,"[1060, 980, 940, 960, 750]","[прокат, отжиг]",980
5,Печь №21,"[1100, 1060, 960, 950]","[ковка, прокат]",950
6,Печь №22,"[1100, 1060, 980, 960, 950]","[ковка, прокат]",1060


In [251]:
df1 = todummy(blank_reset_index, 'operation')
df1

,name,temp,start_temp,ковка,отжиг,прокат
0,Печь №2,"[960, 950, 830, 860, 1060]",1060,1,0,0
1,Печь №3,"[960, 950, 860, 820, 1000]",960,1,0,0
2,Печь №10,"[960, 950, 840, 870]",950,1,0,0
3,Печь №15,"[960, 950, 840, 870]",950,1,0,0
4,Печь №20,"[1060, 980, 940, 960, 750]",980,0,1,1
5,Печь №21,"[1100, 1060, 960, 950]",950,1,0,1
6,Печь №22,"[1100, 1060, 980, 960, 950]",1060,1,0,1


In [252]:
df2 = todummy(blank_reset_index, 'temp')
df2

,name,start_temp,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1060,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,960,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,950,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,950,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,980,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,950,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1060,0,1,1,0,0,0,0,0,0,0,1,1,1


In [259]:
# pd.concat([df1, df2[['1000','1060','1100','750','820','830','840','860','870','940','950','960','980']]])
df_res = df1.merge(df2.drop(columns=['start_temp']), left_on='name', right_on='name', how='inner')
df_res = df_res.drop(columns=['temp'])
df_res

,name,start_temp,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1060,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,960,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,980,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,950,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1060,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [260]:
df_res.to_csv('ovens_done.csv')

In [274]:
df_res

,name,start_temp,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1060,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,960,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,980,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,950,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1060,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [275]:
example_answers = df_res.copy()
example_answers

,name,start_temp,ковка,отжиг,прокат,1000,1060,1100,750,820,830,840,860,870,940,950,960,980
0,Печь №2,1060,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0
1,Печь №3,960,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0
2,Печь №10,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,Печь №15,950,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,Печь №20,980,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5,Печь №21,950,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0
6,Печь №22,1060,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1


In [271]:
from sklearn.preprocessing import MultiLabelBinarizer

def todummy(df, row):
    mlb = MultiLabelBinarizer(sparse_output=True)
    df = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop(row)),
                    index=df.index,
                    columns=mlb.classes_))
    # print(mlb.classes_)
    return df

In [127]:
df = blank_reset_index
from sklearn.compose import make_column_transformer
transformer = make_column_transformer(
    (OneHotEncoder(), ['temp', 'operation']),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
print(transformed_df.head())

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [ ]:
blank_reset_index[['temp', 'operation']]

In [105]:

for i in blank.iterrows():
    print(i)
    if i[0] is not float('nan'):
        blank_non_NaN.loc[ len(df.index )] = i
    # df.loc[ len(df.index )] = [value1, value2, value3, ...]
# blank_non_NaN = blank['Печь' in blank.index]
# blank_non_NaN
blank_non_NaN

(0, Бланк ОМД            Печь
Unnamed: 1           Т-ры
Unnamed: 2          Опер.
Unnamed: 3           от Ø
Unnamed: 4      Нач. т-ра
                  ...    
Unnamed: 98       22-23ч.
Unnamed: 99           NaN
Unnamed: 100          NaN
Unnamed: 101          NaN
Unnamed: 102       23-0ч.
Name: 0, Length: 103, dtype: object)


ValueError: cannot set a row with mismatched columns

In [269]:
# снова json
ovens = pd.json_normalize(data['ovens'])
ovens.info()
# data['ovens']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   start_temp     422 non-null    int64 
 1   working_temps  422 non-null    object
 2   operations     422 non-null    object
dtypes: int64(1), object(2)
memory usage: 10.0+ KB


In [272]:
df1=todummy(ovens, 'working_temps')
df1

,start_temp,operations,950,960,970,980,990,1000,1010,1020,...,1150,1160,1170,1180,1190,1200,1210,1220,1230,1240
0,1220,"[prokat, kovka]",0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
1,990,"[otzhig, kovka]",0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1220,[otzhig],0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1020,[kovka],0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,1020,[otzhig],0,0,1,1,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1240,[kovka],0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
418,1050,[prokat],0,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
419,1140,[kovka],0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
420,950,[prokat],1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [273]:
df2=todummy(df1, 'operations')
df2

,start_temp,950,960,970,980,990,1000,1010,1020,1030,...,1180,1190,1200,1210,1220,1230,1240,kovka,otzhig,prokat
0,1220,0,1,0,0,0,0,0,0,1,...,0,1,0,0,1,1,1,1,0,1
1,990,0,0,0,1,1,0,0,0,1,...,0,1,0,0,0,0,0,1,1,0
2,1220,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1020,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
4,1020,0,0,1,1,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1240,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
418,1050,0,0,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
419,1140,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
420,950,1,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [189]:
series = pd.json_normalize(data['series'])
series

,temperature,operations
0,1080,"[{'name': 'nagrev', 'timing': 245}, {'name': '..."
1,970,"[{'name': 'nagrev', 'timing': 62}, {'name': 'o..."
2,1080,"[{'name': 'nagrev', 'timing': 76}, {'name': 'k..."
3,980,"[{'name': 'nagrev', 'timing': 96}, {'name': 'o..."
4,980,"[{'name': 'nagrev', 'timing': 82}, {'name': 'p..."
...,...,...
2589,960,"[{'name': 'nagrev', 'timing': 281}, {'name': '..."
2590,1060,"[{'name': 'nagrev', 'timing': 248}, {'name': '..."
2591,1000,"[{'name': 'nagrev', 'timing': 182}, {'name': '..."
2592,1090,"[{'name': 'nagrev', 'timing': 204}, {'name': '..."


In [193]:
def foo3(op_list):
    return pd.DataFrame(op_list)

foo3(series.iloc[0, 1])

,name,timing
0,nagrev,245
1,prokat,15


In [209]:
df_full = pd.DataFrame(columns=['name', 'timing', 'num'])
# print(df_full)
for i in range(series.shape[0]):
    # print(i)
    df_cur = foo3(series.iloc[i, 1])
    df_cur['num']=i
    df_full=pd.concat([df_full, df_cur])

df_full

,name,timing,num
0,nagrev,245,0
1,prokat,15,0
0,nagrev,62,1
1,otzhig,15,1
0,nagrev,76,2
...,...,...,...
0,nagrev,204,2592
1,otzhig,15,2592
0,nagrev,274,2593
1,kovka,20,2593


In [230]:
one_series = series[['temperature']]
one_series
for i in range(series.shape[0]):
    for j in range(df_full.index.max()):
        df = df_full[(df_full.index==j)&(df_full.num==i)]
        if df.empty != True:
            # print(df)
            row_name = str(df['name'].iloc[0]) + '_'+str(j)
            # print(row_name)
            one_series[row_name] += pd.Series({i: 1})
        
one_series


KeyError: 'nagrev_0'

In [229]:
one_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2594 entries, 0 to 2593
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperature  2594 non-null   int64  
 1   nagrev_0     1 non-null      float64
 2   prokat_1     1 non-null      float64
 3   otzhig_1     1 non-null      float64
 4   kovka_1      1 non-null      float64
 5   kovka_2      1 non-null      float64
dtypes: float64(5), int64(1)
memory usage: 121.7 KB


In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)
df = blank_reset_index
df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('temp')),
                index=df.index,
                columns=mlb.classes_))
# print(mlb.classes_)
df

In [377]:
# answers
from openpyxl import load_workbook

# Загрузка файла Excel
workbook = load_workbook('ПланОМД.xlsm')

# Выбор нужного листа
sheet = workbook['Бланк']

# Получение цвета ячейки в указанной координате (например, A1)
cell_color = sheet['A1'].fill.start_color.index

print(f"Цвет ячейки A1: {cell_color}")

Цвет ячейки A1: 00000000


In [378]:
dict_answer = {}
for row in range(1, 60):
    for column in range(1, 100):
        # print(sheet.cell(row=row, column=column).value)
        dict_answer[(row, column)]=sheet.cell(row=row, column=column).value
        
dict_answer

{(1, 1): 'Бланк ОМД',
 (1, 2): None,
 (1, 3): None,
 (1, 4): None,
 (1, 5): None,
 (1, 6): None,
 (1, 7): None,
 (1, 8): None,
 (1, 9): None,
 (1, 10): None,
 (1, 11): datetime.datetime(2018, 12, 19, 0, 0),
 (1, 12): None,
 (1, 13): None,
 (1, 14): None,
 (1, 15): None,
 (1, 16): None,
 (1, 17): None,
 (1, 18): None,
 (1, 19): None,
 (1, 20): None,
 (1, 21): None,
 (1, 22): None,
 (1, 23): None,
 (1, 24): None,
 (1, 25): None,
 (1, 26): None,
 (1, 27): None,
 (1, 28): None,
 (1, 29): None,
 (1, 30): None,
 (1, 31): None,
 (1, 32): None,
 (1, 33): None,
 (1, 34): None,
 (1, 35): None,
 (1, 36): None,
 (1, 37): None,
 (1, 38): None,
 (1, 39): None,
 (1, 40): None,
 (1, 41): None,
 (1, 42): None,
 (1, 43): None,
 (1, 44): None,
 (1, 45): None,
 (1, 46): None,
 (1, 47): None,
 (1, 48): None,
 (1, 49): None,
 (1, 50): None,
 (1, 51): None,
 (1, 52): None,
 (1, 53): None,
 (1, 54): None,
 (1, 55): None,
 (1, 56): None,
 (1, 57): None,
 (1, 58): None,
 (1, 59): None,
 (1, 60): None,
 (1, 61):

In [379]:
def get_lists(cur_row):
    temp_list = []
    form_list = []
    detal_list = []
    operation_list = []
    cur_temp = str(sheet.cell(row=cur_row-2, column=7-2).value) + ' C'
    cur_form = str('nan')
    cur_detal = str('nan')
    for column in range(7, 100+7):
        
        
        val = sheet.cell(row=cur_row, column=column).value
        val_form = sheet.cell(row=cur_row-1, column=column).value
        val_detal = sheet.cell(row=cur_row-2, column=column).value
        val_color = sheet.cell(row=5, column=column).fill.start_color.index
        if 'C' in str(val):
            cur_temp = val.replace('C', '').replace('\'', '')
        temp_list += [cur_temp.split()[0]]
        if 'Б' in str(val_form):
            cur_form = val_form
        form_list += [cur_form]
        if val_detal != None:
            cur_detal = val_detal
        detal_list += [cur_detal]
        operation_list += [legend[sheet.cell(row=5, column=column).fill.start_color.index]]
        # print(sheet.cell(row=5, column=column).value)
        # print(sheet.cell(row=5, column=column).fill.start_color.index)
        #dict_answer[(row, column)]=sheet.cell(row=row, column=column).value
        # dict_answer[column] = str(cur_temp).split()[0] + ' ' + legend[sheet.cell(row=5, column=column).fill.start_color.index]
    return temp_list, form_list, detal_list, operation_list

In [380]:
row_curr = []
for o, i, j, k in zip(operation_list, temp_list, form_list, detal_list):
    row_curr += [o, i, j, k]
row_curr

['нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев',
 "1060'C",
 '[440-385]Б.1',
 '3AL-2,5V, 15143-18',
 'нагрев

In [381]:
dop_rows = []
for i in range(1, 101):
    dop_rows += [f'temp_{i}', f'form_{i}', f'detal_{i}', f'operation_{i}']
# print(dop_rows)
df_answer = pd.DataFrame(columns=['печь']+dop_rows)
df_answer

,печь,temp_1,form_1,detal_1,operation_1,temp_2,form_2,detal_2,operation_2,temp_3,...,detal_98,operation_98,temp_99,form_99,detal_99,operation_99,temp_100,form_100,detal_100,operation_100


In [382]:
# df_answer.loc[ len(df_answer.index )] =['печь №1'] + row_curr
# df_answer

In [383]:
def excel_to_list(): 
    fur = ['Печь №2', 'Печь №3', 'Печь №10', 'Печь №15', 'Печь №20', 'Печь №21', 'Печь №22']
    for i, name in zip([5, 11, 17, 23, 29, 35, 41], fur):
        temp_list, form_list, detal_list, operation_list = get_lists(i)
        row_curr = []
        for o, i, j, k in zip(temp_list, form_list, detal_list, operation_list):
            row_curr += [o, i, j, k]
        row_curr
        #print(operation_list)
        df_answer.loc[ len(df_answer.index )] =[name] + row_curr
        
excel_to_list()
df_answer

,печь,temp_1,form_1,detal_1,operation_1,temp_2,form_2,detal_2,operation_2,temp_3,...,detal_98,operation_98,temp_99,form_99,detal_99,operation_99,temp_100,form_100,detal_100,operation_100
0,Печь №2,1060,[440-385]Б.1,"3AL-2,5V, 15143-18",нагрев,1060,[440-385]Б.1,"3AL-2,5V, 15143-18",нагрев,1060,...,"6AL-7Nb, 34-15053-18 №9,11",Nothing,1060,[115-95]Б.,"6AL-7Nb, 34-15053-18 №9,11",Nothing,1060,[115-95]Б.,"6AL-7Nb, 34-15053-18 №9,11",Nothing
1,Печь №3,960,[440-390]Б.,"Э110, 714-18-1слиток",нагрев,960,[440-390]Б.,"Э110, 714-18-1слиток",нагрев,960,...,"Э110, 234-18-1",Nothing,960,[440-120х300]Б.,"Э110, 234-18-1",Nothing,960,[440-120х300]Б.,"Э110, 234-18-1",Nothing
2,Печь №10,950,[235-205]Б.2,"6AL-4V, 34-15023-18 №1",нагрев,950,[235-205]Б.2,"6AL-4V, 34-15023-18 №1",нагрев,950,...,"6AL-4V, 34-8743-18",Nothing,950,[105-105]Б.1,"6AL-4V, 34-8743-18",Nothing,950,[105-105]Б.1,"6AL-4V, 34-8743-18",Nothing
3,Печь №15,950,nan,nan,нагрев,950,[235-205]Б.,"6AL-4V, 34-15023-18",нагрев,950,...,"6AL-4V, 34-8923-18 №1",Nothing,950,[185-165]Б.,"6AL-4V, 34-8923-18 №1",Nothing,950,[185-165]Б.,"6AL-4V, 34-8923-18 №1",Nothing
4,Печь №20,980,nan,nan,нагрев,980,nan,nan,нагрев,980,...,"6AL-4V, 34-8843-18 ч1",Nothing,950,[235-115]Б.,"6AL-4V, 34-8843-18 ч1",Nothing,950,[235-115]Б.,"6AL-4V, 34-8843-18 ч1",Nothing
5,Печь №21,950,nan,nan,нагрев,950,nan,nan,нагрев,950,...,"6AL-4V, 35-150073-18",Nothing,950,[390-360]Б.1,"6AL-4V, 35-150073-18",Nothing,950,[390-360]Б.1,"6AL-4V, 35-150073-18",Nothing
6,Печь №22,1060,nan,nan,нагрев,1060,nan,nan,нагрев,1060,...,"ПТ-7М, 5772-18",Nothing,1060,[440-115]Б.,"ПТ-7М, 5772-18",Nothing,1060,[440-115]Б.,"ПТ-7М, 5772-18",Nothing


In [384]:
df_answer.to_csv('answer_done.csv')

In [304]:
# color: status
legend = {}
for row in range(52, 60):
    legend[sheet.cell(row=row, column=1).fill.start_color.index]=sheet.cell(row=row , column=2).value
    
legend['FFFFD54F'] = 'нагрев'
legend['00000000'] = 'Nothing'
legend

{'FFF47710': 'отжиг',
 'FFFFC000': 'нагрев',
 'FFFFFF00': 'подогрев',
 'FF92D050': 'ковка',
 'FF7030A0': 'прокат',
 'FFC0C0C0': 'прерывание',
 2: 'в ручную - недоступность оборудования (ремонт или предыдущие операции)',
 'FFF2C0C0': 'перевод температуры',
 'FFFFD54F': 'нагрев',
 '00000000': 'Nothing'}